In [72]:
import pandas as pd 
#!pip3 install geopandas
#!pip3 install bokeh

In [73]:
info_vuelos = pd.read_csv('../csv/info_vuelos_full.csv')
info_vuelos.sample(3)

,aerolinea,origen,cod_origen,destino,cod_destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada,distancia_recorrida,precio,asientos_dispobiles
349482,United,Santa Ana,SNA,San Francisco,SFO,"['martes', '19', 'marzo']",08:30,"['martes', '19', 'marzo']",09:30,9580.360780,776.679848,3
48899,Air Canada,Vancouver,CXH,Portland,PTJ,"['sabado', '9', 'marzo']",10:51,"['sabado', '9', 'marzo']",16:09,3724.641531,301.956689,3
168227,American,Jackson,JAN,Charlotte,CLT,"['viernes', '8', 'marzo']",18:30,"['viernes', '8', 'marzo']",21:00,10022.942552,812.559953,4


In [74]:
info_vuelos.sample(2)

,aerolinea,origen,cod_origen,destino,cod_destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada,distancia_recorrida,precio,asientos_dispobiles
75900,AirAsia,Kuala Lumpur,KUL,Johor Bahru,JHB,"['miercoles', '13', 'marzo']",18:10,"['miercoles', '13', 'marzo']",20:35,7662.902351,621.231494,0
255593,Delta,Atlanta,FFC,Augusta,3AU,"['martes', '12', 'marzo']",06:30,"['martes', '12', 'marzo']",07:40,12543.162270,1016.874165,3


In [75]:
import geopandas as gpd

In [76]:
shapefile = '../csv/geo/ne_10m_airports.shp'
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 36 columns):
scalerank     891 non-null int64
featurecla    891 non-null object
type          891 non-null object
name          891 non-null object
abbrev        888 non-null object
location      891 non-null object
gps_code      884 non-null object
iata_code     878 non-null object
wikipedia     885 non-null object
natlscale     891 non-null float64
comments      34 non-null object
wikidataid    890 non-null object
name_ar       377 non-null object
name_bn       101 non-null object
name_de       703 non-null object
name_en       890 non-null object
name_es       752 non-null object
name_fr       730 non-null object
name_el       64 non-null object
name_hi       144 non-null object
name_hu       178 non-null object
name_id       608 non-null object
name_it       524 non-null object
name_ja       865 non-null object
name_ko       371 non-null object
name_nl       630 non-null object
name_

In [77]:
gdf.head(2)

,scalerank,featurecla,type,name,abbrev,location,gps_code,iata_code,wikipedia,natlscale,...,name_pl,name_pt,name_ru,name_sv,name_tr,name_vi,name_zh,wdid_score,ne_id,geometry
0,9,Airport,small,Sahnewal,LUH,terminal,VILD,LUH,http://en.wikipedia.org/wiki/Sahnewal_Airport,8.0,...,None,None,None,Ludhiana Airport,None,None,None,4,1159113785,POINT (75.95707224036518 30.85035985617018)
1,9,Airport,mid,Solapur,SSE,terminal,VASL,SSE,http://en.wikipedia.org/wiki/Solapur_Airport,8.0,...,None,None,None,None,None,None,None,4,1159113803,POINT (75.93305977107549 17.62541518363502)


In [78]:
gdf[gdf.iata_code == 'BOS']

,scalerank,featurecla,type,name,abbrev,location,gps_code,iata_code,wikipedia,natlscale,...,name_pl,name_pt,name_ru,name_sv,name_tr,name_vi,name_zh,wdid_score,ne_id,geometry
839,2,Airport,major,Gen E L Logan Int'l,BOS,terminal,KBOS,BOS,http://en.wikipedia.org/wiki/Logan_Internation...,150.0,...,Port lotniczy Boston,Aeroporto Internacional de Boston,Логан,Logan International Airport,None,Sân bay quốc tế Logan,爱德华·劳伦斯·洛根将军国际机场,4,1159127331,POINT (-71.01640661729584 42.36656581985056)


In [79]:
gdf = gdf[['name', 'iata_code', 'geometry']]
gdf.head(1)

,name,iata_code,geometry
0,Sahnewal,LUH,POINT (75.95707224036518 30.85035985617018)


In [80]:
aeropuertos = info_vuelos['cod_destino'].value_counts().index.tolist()
count = info_vuelos['cod_destino'].value_counts().values.tolist()
d = {'cod_aeropuerto':aeropuertos, 'count':count}
aer = pd.DataFrame(data=d)
aer.head(1)

,cod_aeropuerto,count
0,RYY,17767


In [81]:
merged = gdf.merge(aer, left_on='iata_code', right_on='cod_aeropuerto', how='left')
#Replace NaN values to string 'No data'.
merged.fillna('No data', inplace = True)
merged.head(1)

,name,iata_code,geometry,cod_aeropuerto,count
0,Sahnewal,LUH,POINT (75.95707224036518 30.85035985617018),No data,No data


In [82]:
import json
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [83]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that maps numbers in a range linearly into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = '#d9d9d9')
#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)


In [84]:
#Create figure object.
p = figure(title = 'Aeropuerto mas concurridos', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'per_cent_obesity', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()

Loading BokehJS ...

In [85]:
#Display figure.
show(p)

IsADirectoryError: [Errno 21] Is a directory: '../csv/geo/'